# Combining hydra configurations with `f3dasm`

[hydra](https://hydra.cc/) is an open-source configuration management framework that is widely used in machine learning and other software development domains. It is designed to help developers manage and organize complex configuration settings for their projects, making it easier to experiment with different configurations, manage multiple environments, and maintain reproducibility in their work.

[hydra](https://hydra.cc/) can be seamlessly integrated with the worfklows in f3dasm to manage the configuration settings for the project.

In [1]:
from hydra import compose, initialize

from f3dasm import ExperimentData
from f3dasm.design import Domain

## `Domain` from a [hydra](https://hydra.cc/) configuration file

If you are using [hydra](https://hydra.cc/) to manage your configuration files, you can create a `Domain` from a configuration file. Your config needs to have a key (e.g. `'domain'`) that has two keys: `'input_space'` and `'output_space'`. Each design space dictionary can have parameter names (e.g. `'param_1'`) as keys and a dictionary with an optional parameter type (`'type'`) and the corresponding arguments as values:

```yaml
domain:
  input:
    param_1:
      type: float
      low: -1.0
      high: 1.0
    param_2:
      type: int
      low: 1
      high: 10
    param_3:
      type: category
      categories: ['red', 'blue', 'green', 'yellow', 'purple']
    param_4:
      type: constant
      value: some_value
  output:
    y:
      to_disk: False
```

In order to run the following code snippet, you need to have a configuration file named `'config.yaml'` in the current working directory.

```python
with initialize(version_base=None, config_path="."):
    config = compose(config_name="config")

domain = Domain.from_yaml(config.domain)
```

## `ExperimentData` from a [hydra](https://hydra.cc/) configuration file

If you are using [hydra](https://hydra.cc/) for configuring your experiments, you can use it to construct an `ExperimentData` object from the information in the `'config.yaml'` file with the `from_yaml()` method:

### `ExperimentData` from file

You can create an `ExperimentData` object in the same way as the `from_file()` method, but with the `'from_file'` key in the `'config.yaml'` file:

```yaml
experimentdata:
    from_file: ./example_project_dir
```

```python

with initialize(version_base=None, config_path="."):
    config = compose(config_name="config")


experiment_data = ExperimentData.from_yaml(config.experimentdata)
```

## `ExperimentData` from sampling with [hydra](https://hydra.cc/)

To create the `ExperimentData` object with the `from_sampling()` method, you can use the following configuration:

```yaml
domain:
  input:
    param_1:
      type: float
      low: -1.0
      high: 1.0
    param_2:
      type: int
      low: 1
      high: 10
    param_3:
      type: category
      categories: ['red', 'blue', 'green', 'yellow', 'purple']
    param_4:
      type: constant
      value: some_value
  output:
    y:
      to_disk: False

experimentdata:
    from_sampling:
        domain: ${domain}
        sampler: random
        seed: 1
        n_samples: 1
```

```python

with initialize(version_base=None, config_path="."):
    config = compose(config_name="config")

experiment_data = ExperimentData.from_yaml(config.experimentdata)
```

> You can also combine both approaches to create the `ExperimentData` object by continuing an existing experiment with new samples. This can be done by providing both keys `'from_sampling'` and '`from_file'`